In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
temp_Quito = pd.read_csv('TemperaturasRegistradas/temperature_Quito_celsius.csv')
temp_Oslo = pd.read_csv('TemperaturasRegistradas/temperature_Oslo_celsius.csv')
temp_Melbourne = pd.read_csv('TemperaturasRegistradas/temperature_Melbourne_celsius.csv')

In [3]:
promQuito = temp_Quito.sum()/temp_Quito.count()
promOslo = temp_Oslo.sum()/temp_Oslo.count()
promMelbourne = temp_Melbourne.sum()/temp_Melbourne.count()

In [4]:
i=0
for v in temp_Quito.values.flatten():
    i += (v-promQuito)**2
varianzaQuito = i / (temp_Quito.count()-1)
DesvQuito = math.sqrt(float(varianzaQuito.iloc[0]))
print(DesvQuito)

j=0
for v in temp_Oslo.values.flatten():
    j += (v-promOslo)**2
varianzaOslo = j / (temp_Oslo.count()-1)
DesvOslo = math.sqrt(float(varianzaOslo.iloc[0]))
print(DesvOslo)

k=0
for v in temp_Melbourne.values.flatten():
    k += (v-promMelbourne)**2
varianzaMelbourne = k / (temp_Melbourne.count()-1)
DesvMelbourne = math.sqrt(float(varianzaMelbourne.iloc[0]))
print(DesvMelbourne)



1.301690430933042
9.46169397382391
6.4776540492718455


In [5]:
merged_df = pd.DataFrame({
    'Quito': temp_Quito['AvgTemperature'],
    'Oslo': temp_Oslo['AvgTemperature'],
    'Melbourne': temp_Melbourne['AvgTemperature']
})

print(merged_df)

      Quito  Oslo  Melbourne
0      14.0  -5.0       22.0
1      14.0  -6.0       17.0
2      15.0 -13.0       20.0
3      14.0 -15.0       23.0
4      13.0  -4.0       23.0
...     ...   ...        ...
5839   13.0  -8.0       17.0
5840   14.0 -11.0       19.0
5841   14.0 -18.0       23.0
5842   14.0 -16.0       23.0
5843   14.0 -10.0       24.0

[5844 rows x 3 columns]


In [6]:
def metodoPearson(x, y):
    mean_x = x.sum()/x.count()
    mean_y = y.sum()/y.count()
    
    numerator = np.sum((x - mean_x) * (y - mean_y))
    denominator = np.sqrt(np.sum((x - mean_x)**2) * np.sum((y - mean_y)**2))
    return numerator / denominator

matrizCorr = pd.DataFrame(index=merged_df.columns, columns=merged_df.columns)
for ciudad in merged_df.columns:
    for ciudad2 in merged_df.columns:
        matrizCorr.loc[ciudad, ciudad2] = metodoPearson(merged_df[ciudad], merged_df[ciudad2])

print(matrizCorr)

              Quito      Oslo Melbourne
Quito           1.0  0.066189 -0.053321
Oslo       0.066189       1.0 -0.221326
Melbourne -0.053321 -0.221326       1.0


In [7]:
letters_df = pd.DataFrame(index=merged_df.index, columns=merged_df.columns)

for i in merged_df.columns:
    for j in merged_df.index:
        val = merged_df.loc[j][i]
        if val < 11: letters_df.loc[j, i] = 'F'
        elif val >= 19: letters_df.loc[j, i] = 'C'
        else: letters_df.loc[j, i] = 'T'

print(letters_df)

     Quito Oslo Melbourne
0        T    F         C
1        T    F         T
2        T    F         C
3        T    F         C
4        T    F         C
...    ...  ...       ...
5839     T    F         T
5840     T    F         C
5841     T    F         C
5842     T    F         C
5843     T    F         C

[5844 rows x 3 columns]


In [8]:
ValoresTemp = ['F','T','C']
matriz_contador = np.zeros((len(ValoresTemp), len(ValoresTemp)))

def estado_a_indice(estado):
    return ValoresTemp.index(estado)

def markovOrdenUno(x):
    for i in range(len(letters_df[x]) - 1):
        actual = estado_a_indice(letters_df[x][i])
        siguiente = estado_a_indice(letters_df[x][i+1])
        matriz_contador[actual, siguiente] += 1

    matriz_transicion = np.zeros((len(ValoresTemp), len(ValoresTemp)))
    for i in range(len(ValoresTemp)):
        total = np.sum(matriz_contador[i])
        if total > 0:
            matriz_transicion[i] = matriz_contador[i] / total

    matriz_df = pd.DataFrame(matriz_transicion, index=ValoresTemp, columns=ValoresTemp)
    return matriz_transicion


#Direccion ascendente, de indice fila a indice columna.
print('Caso Quito:')
print(markovOrdenUno('Quito'))
print()


print('Caso Oslo:')
print(markovOrdenUno('Oslo'))
print()

print('Caso Melbourne:')
print(markovOrdenUno('Melbourne'))

Caso Quito:
[[5.40540541e-02 9.45945946e-01 0.00000000e+00]
 [6.03240262e-03 9.93278180e-01 6.89417442e-04]
 [0.00000000e+00 1.00000000e+00 0.00000000e+00]]

Caso Oslo:
[[0.95103858 0.04896142 0.        ]
 [0.02660575 0.96506316 0.00833109]
 [0.         0.31       0.69      ]]

Caso Melbourne:
[[0.92836946 0.06998115 0.00164939]
 [0.02856044 0.92826233 0.04317723]
 [0.0024255  0.15557866 0.84199584]]


In [9]:
ValoresCiudades = ['Quito', 'Oslo', 'Melbourne']

def OrdenCero(temperatura, ciudad):
    val = 0
    total = 0
    for fila in letters_df.index:
        total += 1
        if letters_df.loc[fila, ciudad] == temperatura:
            val += 1
    return val / total

def DistribucionPorCiudad(ciudad):
    distribucion = {}
    for simbolo in ValoresTemp:
        distribucion[simbolo] = OrdenCero(simbolo, ciudad)
    return distribucion

# Mostrar resultados
for ciudad in ValoresCiudades:
    print('Caso', ciudad, ':')
    dist = DistribucionPorCiudad(ciudad)
    for simbolo in dist:
        print(simbolo, ':', dist[simbolo])
    print()


def distribucionConjunta(ciudad):
    dist = DistribucionPorCiudad(ciudad)
    acumulado = 0
    V0acum = [0] * len(ValoresTemp)
    for simbolo in dist:
        acumulado = acumulado + dist[simbolo]
        V0acum.append(acumulado)
    return V0acum

def distribucionConjuntaTransicion(ciudad):
    matriz_transicion = markovOrdenUno(ciudad)
    macum = []
    for fila in matriz_transicion:
        fila_acum = []
        suma = 0
        for valor in fila:
            suma += valor
            fila_acum.append(suma)
        macum.append(fila_acum)
    return macum
            


Caso Quito :
F : 0.006331279945242984
T : 0.992984257357974
C : 0.0006844626967830253

Caso Oslo :
F : 0.6858316221765913
T : 0.2806297056810404
C : 0.03353867214236824

Caso Melbourne :
F : 0.034223134839151265
T : 0.5059890485968515
C : 0.45978781656399725



In [14]:
import random

simbolos = [1,2,3]
V0acum = distribucionConjunta("Quito")   #calcular en base al 2.1
Macum = distribucionConjuntaTransicion("Quito")

MINMENSAJES = 100
E = 0.0001

def primer_simbolo():
    r = random.random()
    for i in range (len(simbolos)):
        if r < V0acum[i]:
            return i

def siguiente_simbolo(origen: int):
    r = random.random()
    for i in range (len(simbolos)):
        if r < Macum[origen] [i]:
            return i

def converge(estado_act: list, estado_ant: list):
    for i in range(len(estado_act)):
        if (abs(estado_act[i] - estado_ant[i]) > E):
            return 0
    return 1

def estadoestacionario():
    conteo = [0,0,0]
    estado_ant = [-1, -1, -1]
    estado_act = [0, 0, 0]
    mensajes = 0
    #s = primer_simbolo()
    s = 1
    while not converge(estado_act, estado_ant) or (mensajes < MINMENSAJES):
        s = siguiente_simbolo(s)
        conteo[s] = conteo[s] + 1
        mensajes = mensajes + 1
        estado_ant = estado_act.copy()
        estado_act = [x / mensajes for x in conteo]
    print(estado_act)
    return estado_act

estadoestacionario()

TIEMPOMINIMO = 1

def media_primera_recurrencia(simbolo: int):
    retornos = 0
    media = 0
    media_anterior = -1
    t_actual = 0
    s = simbolo
    while not converge(media, media_anterior) or (t_actual < TIEMPOMINIMO):
        s = siguiente_simbolo(s)
        t_actual = t_actual + 1
        if (s == simbolo):
            retornos = retornos + 1
            media_anterior = media
            media = t_actual/retornos
    return media

[0.11310494834148994, 0.8162044589450789, 0.07069059271343121]


In [11]:
for ciudad in ValoresCiudades:
    val = 0
    dist = DistribucionPorCiudad(ciudad)
    for i in dist:
        val += -dist[i] * np.log2(dist[i])
    print('Entropia para ', ciudad, ': ', val)

Entropia para  Quito :  0.06352072242389746
Entropia para  Oslo :  1.0518836866635417
Entropia para  Melbourne :  1.179329303672061


In [12]:
for ciudad in ValoresCiudades:
    val = 0
    matr = markovOrdenUno(ciudad)
    for i in matr.index:
        for j in matr.columns:
            val += -matr[i][j] * np.log2(matr[i][j])
    print('Entropia para', ciudad, ': ', val)

AttributeError: 'numpy.ndarray' object has no attribute 'index'